#💾 **Importando JSON.**

##📌 Bibliotecas.

In [155]:
import pandas as pd
import requests
from pandas import json_normalize
import numpy as np
import re

In [156]:
url = "https://raw.githubusercontent.com/EuniceGarcia503/Challenge2_TelecomX_LATAM/refs/heads/main/TelecomX_Data.json"
response = requests.get(url)
data = response.json()
df = pd.DataFrame(data)
df.head()

,customerID,Churn,customer,phone,internet,account
0,0002-ORFBO,No,"{'gender': 'Female', 'SeniorCitizen': 0, 'Part...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'DSL', 'OnlineSecurity': '...","{'Contract': 'One year', 'PaperlessBilling': '..."
1,0003-MKNFE,No,"{'gender': 'Male', 'SeniorCitizen': 0, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'Yes'}","{'InternetService': 'DSL', 'OnlineSecurity': '...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
2,0004-TLHLJ,Yes,"{'gender': 'Male', 'SeniorCitizen': 0, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'Fiber optic', 'OnlineSecu...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
3,0011-IGKFF,Yes,"{'gender': 'Male', 'SeniorCitizen': 1, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'Fiber optic', 'OnlineSecu...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
4,0013-EXCHZ,Yes,"{'gender': 'Female', 'SeniorCitizen': 1, 'Part...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'Fiber optic', 'OnlineSecu...","{'Contract': 'Month-to-month', 'PaperlessBilli..."



*   customerID: número de identificación único de cada cliente
*   Churn: si el cliente dejó o no la empresa
gender: género (masculino y femenino)
*   SeniorCitizen: información sobre si un cliente tiene 65 años o más
*   Partner: si el cliente tiene pareja o no
*   Dependents: si el cliente tiene dependientes o no
*   tenure: meses de contrato del cliente
*   PhoneService: suscripción de servicio telefónico
*   MultipleLines: suscripción a más de una línea de teléfono
*   InternetService: suscripción a un proveedor de internet
*   OnlineSecurity: suscripción adicional de seguridad online
*   OnlineBackup: suscripción adicional de backup online
*   DeviceProtection: suscripción adicional de protección en el dispositivo
*   TechSupport: suscripción adicional de soporte técnico, menos tiempo de espera
*   StreamingTV: suscripción de TV por cable
*   StreamingMovies: suscripción de streaming de películas
*   Contract: tipo de contrato
*   PaperlessBilling: si el cliente prefiere recibir la factura online
*   PaymentMethod: forma de pago
*   Charges.Monthly: total de todos los servicios del cliente por mes
*   Charges.Total: total gastado por el cliente

#🧰 **Transformación.**

##🛠️ **Normalización de Columnas.**

In [157]:
df_telecomX_normalizado = pd.json_normalize(data)
df_telecomX_normalizado.head()

,customerID,Churn,customer.gender,customer.SeniorCitizen,customer.Partner,customer.Dependents,customer.tenure,phone.PhoneService,phone.MultipleLines,internet.InternetService,...,internet.OnlineBackup,internet.DeviceProtection,internet.TechSupport,internet.StreamingTV,internet.StreamingMovies,account.Contract,account.PaperlessBilling,account.PaymentMethod,account.Charges.Monthly,account.Charges.Total
0,0002-ORFBO,No,Female,0,Yes,Yes,9,Yes,No,DSL,...,Yes,No,Yes,Yes,No,One year,Yes,Mailed check,65.6,593.3
1,0003-MKNFE,No,Male,0,No,No,9,Yes,Yes,DSL,...,No,No,No,No,Yes,Month-to-month,No,Mailed check,59.9,542.4
2,0004-TLHLJ,Yes,Male,0,No,No,4,Yes,No,Fiber optic,...,No,Yes,No,No,No,Month-to-month,Yes,Electronic check,73.9,280.85
3,0011-IGKFF,Yes,Male,1,Yes,No,13,Yes,No,Fiber optic,...,Yes,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,98.0,1237.85
4,0013-EXCHZ,Yes,Female,1,Yes,No,3,Yes,No,Fiber optic,...,No,No,Yes,Yes,No,Month-to-month,Yes,Mailed check,83.9,267.4


##🛠️ **Identificación de Inconsistencias en Datos.**

In [158]:
# Código para Verificación de Valores Únicos por Columna.
print(f"📝Resumen para Verificación de Valores Únicos por Columna.\n")
for col in df_telecomX_normalizado.columns:
  print(f"🟣 Columna: '{col}': {df_telecomX_normalizado[col].nunique()}\n")

  if df_telecomX_normalizado[col].nunique() < 50: # Imprima los valores solamente si no hay muchos
    print(f"\t✏️ valores = {df_telecomX_normalizado[col].unique()}\n")
  print('-' * 50)  # Línea divisoria para mejor visualización

📝Resumen para Verificación de Valores Únicos por Columna.

🟣 Columna: 'customerID': 7267

--------------------------------------------------
🟣 Columna: 'Churn': 3

	✏️ valores = ['No' 'Yes' '']

--------------------------------------------------
🟣 Columna: 'customer.gender': 2

	✏️ valores = ['Female' 'Male']

--------------------------------------------------
🟣 Columna: 'customer.SeniorCitizen': 2

	✏️ valores = [0 1]

--------------------------------------------------
🟣 Columna: 'customer.Partner': 2

	✏️ valores = ['Yes' 'No']

--------------------------------------------------
🟣 Columna: 'customer.Dependents': 2

	✏️ valores = ['Yes' 'No']

--------------------------------------------------
🟣 Columna: 'customer.tenure': 73

--------------------------------------------------
🟣 Columna: 'phone.PhoneService': 2

	✏️ valores = ['Yes' 'No']

--------------------------------------------------
🟣 Columna: 'phone.MultipleLines': 3

	✏️ valores = ['No' 'Yes' 'No phone service']

------------

In [159]:
# Verificando si hay duplicados
print(f"📝 Resumen de Verificación de Datos duplicados y Núlos.\n")
duplicados = df_telecomX_normalizado.duplicated().sum()
print(f"🟣 Número de líneas duplicadas:\n")

if  duplicados:
    print(f"\t🔸{duplicados}")
else:
    print("   ✅ No hay registros duplicados.")


# Verificando si hay valores Nulos
valores_nulos = df_telecomX_normalizado.isnull().sum()
valores_nulos.name = 'Valores Nulos'
valores_nulos_filtrados = valores_nulos[valores_nulos > 0]

print(f"\n🟣 Valores nulos por columna:\n")
if not valores_nulos_filtrados.empty:
    for columna, cantidad in valores_nulos_filtrados.items():
        print(f"\t🔸 {columna}: {cantidad}")
else:
    print("   ✅ No hay columnas con datos nulos.\n")
    display(valores_nulos)


📝 Resumen de Verificación de Datos duplicados y Núlos.

🟣 Número de líneas duplicadas:

   ✅ No hay registros duplicados.

🟣 Valores nulos por columna:

   ✅ No hay columnas con datos nulos.



,Valores Nulos
customerID,0
Churn,0
customer.gender,0
customer.SeniorCitizen,0
customer.Partner,0
customer.Dependents,0
customer.tenure,0
phone.PhoneService,0
phone.MultipleLines,0
internet.InternetService,0


In [160]:
#código para calcular valores sin registro
valores_blanco_vacio = df_telecomX_normalizado.apply(lambda x: x.astype(str).str.strip() == '').sum()
valores_blanco_vacio.name = 'Registro en blanco'
print("\n📝 Resumen para Registros en Blanco por Columna.\n")
display(valores_blanco_vacio)


📝 Resumen para Registros en Blanco por Columna.



,Registro en blanco
customerID,0
Churn,224
customer.gender,0
customer.SeniorCitizen,0
customer.Partner,0
customer.Dependents,0
customer.tenure,0
phone.PhoneService,0
phone.MultipleLines,0
internet.InternetService,0


In [171]:
df_telecomX_normalizado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7267 entries, 0 to 7266
Data columns (total 21 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   customerID                 7267 non-null   object 
 1   Churn                      7267 non-null   object 
 2   customer.gender            7267 non-null   object 
 3   customer.SeniorCitizen     7267 non-null   int64  
 4   customer.Partner           7267 non-null   object 
 5   customer.Dependents        7267 non-null   object 
 6   customer.tenure            7267 non-null   int64  
 7   phone.PhoneService         7267 non-null   object 
 8   phone.MultipleLines        7267 non-null   object 
 9   internet.InternetService   7267 non-null   object 
 10  internet.OnlineSecurity    7267 non-null   object 
 11  internet.OnlineBackup      7267 non-null   object 
 12  internet.DeviceProtection  7267 non-null   object 
 13  internet.TechSupport       7267 non-null   objec

In [161]:
#1. Columna 'SeniorCitizen'(0,1)(objet)Se puede convertir sus valores a 'Yes' o 'No'
#2. Columnas 'Contract y 'PaymentMethod' podríamos unir palabras por un guion (-) para tokenizar
#3. Columna 'PaymentMethod' remover (), reemplazar espacios por guion.
#4. Columna 'Charges.Total' convertir a tipo float64.
#5. Columnas 'Churn' ''
#6. Tratamiento preventivo para columnas tipo texto: Remover espacios al inicio y final,convertimos a minúscula, removemos valores nulos por desconocido.
#7. Tratamiento preventivo para columnas con caracteres numericos y conversión tipo float64.

##🛠️ **Manejo de Inconsistencias en Datos.**

In [173]:
#Tratamiento mínimo para columnas tipo cadena de texto.
#Remover espacios al inicio y final,convertimos a minúscula, removemos valores nulos por desconocido.
df_telecomX_normalizado = df_telecomX_normalizado.apply(lambda x: x.astype(str).str.strip().str.lower().replace('', np.nan).fillna('desconocido'))

In [175]:
#Tratamiento mínimo para columnas tipo cadena de texto.
#Tratamiento de datos en columna 'SeniorCitizen'(0,1)(objet)Se puede convertir sus valores a 'Yes' o 'No'
df_telecomX_normalizado['customer.SeniorCitizen'] = df_telecomX_normalizado['customer.SeniorCitizen'].astype('str').replace('0','No').replace('1','Yes')

In [177]:
#Columnas 'Contract y 'PaymentMethod' podríamos unir palabras por un guion (-) para tokenizar.
#Son las columnas cuyos registros deseamos unir con un guion.
columnas_espacios = ['account.Contract','account.PaymentMethod','internet.InternetService']
for col in columnas_espacios:
  df_telecomX_normalizado[col] = df_telecomX_normalizado[col].str.replace(' ','-')

In [178]:
#Columna 'PaymentMethod' remover (), reemplazar espacios por guion.
df_telecomX_normalizado['account.PaymentMethod'] = df_telecomX_normalizado['account.PaymentMethod'] \
    .str.replace('(', '-', regex=False) \
    .str.replace(')', '', regex=False) \
    .str.replace(' ', '-', regex=False)

In [179]:
#Columna 'Charges.Monthly','Charges.Total' Limpieza preventiva de caracteres menos numericos y el punto.
columnas_numericas = ['account.Charges.Monthly','account.Charges.Total']
for col in columnas_numericas:
  df_telecomX_normalizado[col] = pd.to_numeric(df_telecomX_normalizado[col].astype(str).str.replace(r'[^0-9.]', '', regex=True), errors='coerce')

In [166]:
df_telecomX_normalizado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7267 entries, 0 to 7266
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7267 non-null   object 
 1   Churn             7267 non-null   object 
 2   gender            7267 non-null   object 
 3   SeniorCitizen     7267 non-null   object 
 4   Partner           7267 non-null   object 
 5   Dependents        7267 non-null   object 
 6   tenure            7267 non-null   int64  
 7   PhoneService      7267 non-null   object 
 8   MultipleLines     7267 non-null   object 
 9   InternetService   7267 non-null   object 
 10  OnlineSecurity    7267 non-null   object 
 11  OnlineBackup      7267 non-null   object 
 12  DeviceProtection  7267 non-null   object 
 13  TechSupport       7267 non-null   object 
 14  StreamingTV       7267 non-null   object 
 15  StreamingMovies   7267 non-null   object 
 16  Contract          7267 non-null   object 


#📊 Carga y análisis

In [180]:
df_telecomX_datos_copy = df_telecomX_normalizado.copy()

In [169]:
df_telecomX_datos_copy.head()

,customerID,Churn,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,...,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,Charges.Monthly,Charges.Total
0,0002-ORFBO,no,female,no,yes,yes,9,yes,no,dsl,...,yes,no,yes,yes,no,one-year,yes,mailed-check,65.6,593.30
1,0003-MKNFE,no,male,no,no,no,9,yes,yes,dsl,...,no,no,no,no,yes,month-to-month,no,mailed-check,59.9,542.40
2,0004-TLHLJ,yes,male,no,no,no,4,yes,no,fiber-optic,...,no,yes,no,no,no,month-to-month,yes,electronic-check,73.9,280.85
3,0011-IGKFF,yes,male,yes,yes,no,13,yes,no,fiber-optic,...,yes,yes,no,yes,yes,month-to-month,yes,electronic-check,98.0,1237.85
4,0013-EXCHZ,yes,female,yes,yes,no,3,yes,no,fiber-optic,...,no,no,yes,yes,no,month-to-month,yes,mailed-check,83.9,267.40


#📄Informe final